# Dataset_Preprocessing

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Uniform Dataset Preprocessing](#transformations)

## 1. Libraries <a id = "libraries"> 

In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## 2. Gridworld Creation <a id = "gridworld">

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

## 4. Uniform Dataset Preprocessing <a id = "transformations">

In [11]:
df = pd.read_csv("DatasetBefore.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)   
df

,Current_State,Pit_Position,Goal_Position,Wall_Position,Current_Reward,Q1_UP,Q2_DOWN,Q3_LEFT,Q4_RIGHT
0,0,0,2,1,-10,2.614309,2.558848,3.948454,4.814851
1,2,0,2,1,10,6.871885,6.282998,6.243661,6.415882
2,3,0,2,1,-1,7.459104,5.998907,8.490587,6.589260
3,4,0,2,1,-1,0.305944,1.363386,3.014956,4.630818
4,5,0,2,1,-1,3.776497,4.053163,3.613774,8.103388
...,...,...,...,...,...,...,...,...,...
50395,10,15,13,14,-1,3.689109,7.530389,7.412851,6.018137
50396,11,15,13,14,-1,1.637386,3.614807,5.358669,4.984922
50397,12,15,13,14,-1,5.306869,7.611459,9.131452,12.008292
50398,13,15,13,14,10,7.040803,8.193069,7.585652,8.199212


### Rearrangement of dataset

In [12]:
counter = 0

h = 0
w, h = 6, 3000000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(df["Current_State"][i])
    game.board.components['Goal'].pos = from1dto2d(df["Goal_Position"][i])
    game.board.components['Wall'].pos = from1dto2d(df["Wall_Position"][i])
    game.board.components['Pit'].pos = from1dto2d(df["Pit_Position"][i])
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q1_UP"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('d')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q2_DOWN"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('u')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q3_LEFT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('r')
        counter += 1 
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))   
    
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
    if p == pi:
        pass
    else:
        matrix[counter][0] = from2dto1d((str(game.board.components['Player'].pos)))
        matrix[counter][1] = from2dto1d((str(game.board.components['Goal'].pos)))
        matrix[counter][2] = from2dto1d((str(game.board.components['Wall'].pos)))
        matrix[counter][3] = from2dto1d((str(game.board.components['Pit'].pos)))
        matrix[counter][4] = df["Q4_RIGHT"][i]
        reward = game.reward()
        matrix[counter][5] = reward
        game.makeMove('l')
        counter += 1 

In [13]:
column_names = ["Player",  "Goal", "Wall", "Pit", "Q_value", "Reward"]
df_matrix = pd.DataFrame(matrix, columns = column_names)
df_matrix = df_matrix.loc[(df_matrix != 0).any(axis=1)]
df_matrix

,Player,Goal,Wall,Pit,Q_value,Reward
0,4,2,1,0,2.558848,-1
1,6,2,1,0,6.282998,-1
2,3,2,1,0,6.415882,-1
3,7,2,1,0,5.998907,-1
4,2,2,1,0,8.490587,10
...,...,...,...,...,...,...
141115,8,13,14,15,5.306869,-1
141116,13,13,14,15,12.008292,10
141117,9,13,14,15,7.040803,-1
141118,12,13,14,15,7.585652,-1


In [18]:
# Drop duplicate positions
df_new = df_matrix.drop_duplicates(keep='last',subset=[ 'Player', 'Goal', 'Wall', 'Pit'])
df_new = df_new.reset_index()
df_new.drop('index', axis=1, inplace=True)   
df_new

,Player,Goal,Wall,Pit,Q_value,Reward
0,0,2,1,0,0.305944,-10
1,2,2,1,0,9.047926,10
2,3,2,1,0,7.594888,-1
3,4,2,1,0,2.538072,-1
4,5,2,1,0,5.857223,-1
...,...,...,...,...,...,...
50395,8,13,14,15,5.306869,-1
50396,13,13,14,15,12.008292,10
50397,9,13,14,15,7.040803,-1
50398,12,13,14,15,7.585652,-1


In [19]:
df_new.to_csv('DatasetAfter1.csv')

## Bellman Operator Dataset Creation

In [20]:
no_dup_df = df_new
no_dup_df

,Player,Goal,Wall,Pit,Q_value,Reward
0,0,2,1,0,0.305944,-10
1,2,2,1,0,9.047926,10
2,3,2,1,0,7.594888,-1
3,4,2,1,0,2.538072,-1
4,5,2,1,0,5.857223,-1
...,...,...,...,...,...,...
50395,8,13,14,15,5.306869,-1
50396,13,13,14,15,12.008292,10
50397,9,13,14,15,7.040803,-1
50398,12,13,14,15,7.585652,-1


In [23]:
counter = 0

h = 0
w, h = 7, 300000
matrix = [[0 for x in range(w)] for y in range(h)] 

for i in range (0, no_dup_df.shape[0]):
    game = Gridworld(size=4, mode='random')
    game.board.components['Player'].pos = from1dto2d(no_dup_df["Player"][i])
    game.board.components['Goal'].pos = from1dto2d(no_dup_df["Goal"][i])
    game.board.components['Wall'].pos = from1dto2d(no_dup_df["Wall"][i])
    game.board.components['Pit'].pos = from1dto2d(no_dup_df["Pit"][i])
    
    q_value = []
    reward_arr = []
    
    pi = from2dto1d((str(game.board.components['Player'].pos)))
    
    goal = int(no_dup_df["Goal"][i])
    pit = int(no_dup_df["Pit"][i])
    wall = int(no_dup_df["Wall"][i])
    
    matrix[counter][0] = pi
    matrix[counter][1] = no_dup_df["Goal"][i]
    matrix[counter][2] = no_dup_df["Wall"][i]
    matrix[counter][3] = no_dup_df["Pit"][i]
    matrix[counter][4] = no_dup_df["Q_value"][i]
    matrix[counter][5] = no_dup_df["Reward"][i]
    
    game.makeMove('u')
    p = from2dto1d((str(game.board.components['Player'].pos)))

    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == int(p)) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('d')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('d')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('u')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))    
    game.makeMove('l')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('r')
        
        
    pi = from2dto1d((str(game.board.components['Player'].pos)))       
    game.makeMove('r')
    p = from2dto1d((str(game.board.components['Player'].pos)))
            
    if p == pi:
        q_value.append(-10)
        reward_arr.append(-100)
    else:
        q_value.append(no_dup_df[(no_dup_df["Player"] == p) & (no_dup_df["Goal"] == goal)  & (no_dup_df["Wall"] == wall) & (no_dup_df["Pit"] == pit)]['Q_value'].values[0])
        reward = game.reward()
        reward_arr.append(reward)
        game.makeMove('l')
        
    
    max_qvalue = q_value[np.argmax(q_value)]
    reward_max = reward_arr[np.argmax(q_value)]
    bellman = max_qvalue*0.9 + reward_max
    if pi == goal or bellman > 10:
        matrix[counter][6] = 10
    else:
        matrix[counter][6] = bellman
    counter += 1

In [24]:
import pandas as pd
column_names = ["Player","Goal", "Wall", "Pit", "Q_Current", "Reward_Current", "Bellman_Operator"
                ]
bell_df = pd.DataFrame(matrix, columns = column_names)
bell_df = bell_df.loc[(bell_df != 0).any(axis=1)]
bell_df

,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,2,1,0,0.305944,-10,1.284265
1,2,2,1,0,9.047926,10,10.000000
2,3,2,1,0,7.594888,-1,10.000000
3,4,2,1,0,2.538072,-1,4.271501
4,5,2,1,0,5.857223,-1,5.000492
...,...,...,...,...,...,...,...
50395,8,13,14,15,5.306869,-1,5.827087
50396,13,13,14,15,12.008292,10,10.000000
50397,9,13,14,15,7.040803,-1,10.000000
50398,12,13,14,15,7.585652,-1,10.000000


In [25]:
bell_df.to_csv('DatasetAfter2.csv')